If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8007/tree?token=encode-images```  

This notebook describes a pipeline to encode panoramas derived from Naik et al (2014).  
It takes:
- a base image as a ```.jpg```;
- its GIST descriptor as a ```.csv```;
- its textonmap as a ```.png```; and
returns a 3736×1 matrix as a ```.json``` encoding the image information.  

More specifically:
1. The RGB histogram of the textonmap is calculated yielding a 8×8×8 histogram;
2. The CIELAB historgram of the base image is calculated yielding a 14×4×4 histogram;
3. Flattening the 3000×1 GIST discriptor with a 8×8×8 matrix and a 14×4×4 matrix yields a 3736×1 matrix.  

> N. Naik, J. Philipoom, R. Raskar and C. Hidalgo, "Streetscore -- Predicting the Perceived Safety of One Million Streetscapes," 2014 IEEE Conference on Computer Vision and Pattern Recognition Workshops, Columbus, OH, USA, 2014, pp. 793-799, doi: 10.1109/CVPRW.2014.121.

In [1]:
target_dir = "data"
image_dir = "shunfu-panos"
gist_dir = "shunfu-gists"
textonmap_dir = "shunfu-textonmaps"
output_dir = "shunfu-panos-encoded"

In [2]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import csv
import json
import os
from pathlib import Path

In [ ]:
images_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, image_dir)):
    images_path.extend(filenames)
    break
images_id = {}
for image_path in images_path:
   images_id['.'.join(image_path.split('.')[:-1])] = os.path.join(target_dir, image_dir, image_path)
images_df = pd.DataFrame.from_dict(images_id, orient="index", columns=["fp"])

gists_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, gist_dir)):
    gists_path.extend(filenames)
    break
gists_id = {}
for gist_path in gists_path:
   gists_id['.'.join(gist_path.split('.')[:-1])] = os.path.join(target_dir, gist_dir, gist_path)
gists_df = pd.DataFrame.from_dict(gists_id, orient="index", columns=["fp"])

textonmaps_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, textonmap_dir)):
    textonmaps_path.extend(filenames)
    break
textonmaps_id = {}
for textonmap_path in textonmaps_path:
   textonmaps_id['.'.join(textonmap_path.split('.')[:-1])] = os.path.join(target_dir, textonmap_dir, textonmap_path)
textonmaps_df = pd.DataFrame.from_dict(textonmaps_id, orient="index", columns=["fp"])

ids_df = images_df.join(
    gists_df, how="inner", lsuffix="_images", rsuffix="_gists").join(
        textonmaps_df, how="inner")..rename(columns={"fp": "fp_textonmaps"})

In [ ]:
Path(os.path.join(target_dir, output_dir)).mkdir(parents=True, exist_ok=True)
count = 0
for index, row in ids_df.iterrows():
    try:
        image = cv2.imread(row["fp_images"], cv2.IMREAD_COLOR)
        gist = []
        with open(row["fp_gists"], 'r') as fp:
            csv_reader = csv.reader(fp)
            for line in csv_reader:
                if line:
                    gist.append(float(line[0]))
        area = sum(gist)
        gist = [x / area for x in gist]
        textonmap = cv2.imread(row["fp_textonmaps"], cv2.IMREAD_COLOR)
    except Exception as e:
        print(e)
        continue

    image = cv2.cvtColor(cv2.resize(image, (512, 256)), cv2.COLOR_BGR2LAB)
    textonmap = cv2.cvtColor(cv2.resize(textonmap, (512, 256)), cv2.COLOR_BGR2RGB)

    texton_histogram, e = np.histogramdd(textonmap.reshape(256 * 512, 3), bins=[8, 8, 8], range=[(0, 256), (0, 256), (0, 256)])
    colour_histogram, e = np.histogramdd(image.reshape(256 * 512, 3), bins=[14, 4, 4], range=[(0, 256), (0, 256), (0, 256)])

    image_embedding = np.array(texton_histogram).flatten().tolist()+ gist + np.array(colour_histogram).flatten().tolist()

    with open(os.path.join(target_dir, output_dir, index + ".json"), 'w') as fp:
        json.dump(image_embedding, fp)

    count += 1
    if not count % 50:
        print(count)